In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

falseGroup = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/"
trueGroup  = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/"

# Get the list of file paths
false_images = [os.path.join(falseGroup, fname) for fname in os.listdir(falseGroup)]
true_images = [os.path.join(trueGroup, fname) for fname in os.listdir(trueGroup)]

# Create labels
false_labels = [0] * len(false_images)  # Label '0' for no tumor
true_labels = [1] * len(true_images)    # Label '1' for tumor present

# Combine the lists of file paths and labels
file_paths = false_images + true_images
labels = false_labels + true_labels

# Split into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Define a function to load and preprocess images
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, tf.cast(label, tf.float32)

# Create training and validation datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(load_image).batch(32).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=len(X_train))

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(load_image).batch(32, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=len(X_val))


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Model creation functions
def create_resnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_vgg_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mobilenet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_custom_cnn(input_shape=(224, 224, 3), num_classes=1):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model evaluation function
def evaluate_model(model, train_dataset, val_dataset):
    # Start the training timer
    start_time = time.time()
    
    # Train the model
    model.fit(train_dataset, epochs=10, validation_data=val_dataset)
    
    # Stop the timer for training time
    train_time = time.time() - start_time
    
    # Predict on validation data
    y_pred = model.predict(val_dataset)
    y_pred = (y_pred > 0.5).astype(int).flatten()
    
    # Collect all true labels from val_dataset
    y_true = tf.concat([labels for _, labels in val_dataset], axis=0).numpy().flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    
    # Calculate inference time
    inference_start = time.time()
    _ = model.predict(val_dataset)
    inference_time = (time.time() - inference_start) / len(val_dataset)
    
    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "auc": auc,
            "train_time": train_time,
            "inference_time": inference_time
           }

# Prepare to run evaluation on all models
model_functions = {
    # 'ResNet50': create_resnet_model,
    # 'VGG16': create_vgg_model,
    # 'EfficientNetB0': create_efficientnet_model,
    # 'MobileNetV2': create_mobilenet_model,
    'Custom CNN': create_custom_cnn
}

results = {}

# Iterate through the models and evaluate each one
for model_name, model_function in model_functions.items():
    print(f"Training and evaluating {model_name}...")
    model = model_function()  # Create the model
    results[model_name] = evaluate_model(model, train_dataset, val_dataset)

# Display the results
df_results = pd.DataFrame(results).T  # Transpose for easier viewing
print(df_results)  # Display the DataFrame in console

Training and evaluating Custom CNN...


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5669 - loss: 1.0223 - val_accuracy: 0.7255 - val_loss: 0.5904
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6801 - loss: 0.6046 - val_accuracy: 0.7647 - val_loss: 0.6120
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8158 - loss: 0.4602 - val_accuracy: 0.7451 - val_loss: 0.6695
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8329 - loss: 0.4130 - val_accuracy: 0.7255 - val_loss: 0.6269
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8520 - loss: 0.3452 - val_accuracy: 0.7255 - val_loss: 0.6580
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8432 - loss: 0.3333 - val_accuracy: 0.7647 - val_loss: 0.7422
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9105 - loss: 0.2216 - val_accuracy: 0.7451 - val_loss: 0.7177
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8944 - loss: 0.2174 - val_accuracy: 0.8039 - val_loss: 0.7603
Epoch 9/